# Exemple Dash Trading OPA

In [1]:
from Binance import Bot_Trading_OPA as Bot

import plotly.express as px
import plotly.graph_objs as go

from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, State, dash_table
import dash_bootstrap_components as dbc


#### Définition des Fonctions "Utilitaires" au Dash

In [2]:
from datetime import datetime, timedelta
import pandas as pd

"""
Step 1 :  Définition des fonctions à crééer dans un fichier "Utilitaire_Dash" indépendant 
          qui pourront être importé par la suite dans l'appli DASH
"""

# -->
def Get_Dash_ListePaire():
    X = Bot.Get_API_Binance().exchange_info()
    return [{'value' : symbol['symbol'], 'label' : symbol['baseAsset'] + '/'+ symbol['quoteAsset'] } for symbol in X['symbols'] ]



def Get_Dash_GraphAcceuil(paire, Période_Debut= None, Période_Fin = None):
    
    df = Bot.Get_Live_InfoPaire(paire, Periode_Debut = Période_Debut, Periode_Fin = Période_Fin )
    
     # Convert the 'OpenTime' column from a timestamp to a datetime object
    df['OpenTime'] = pd.to_datetime(df['OpenTime'], unit='ms')
    
    fig = go.Figure(data=go.Scatter(x=df['OpenTime'], y=df['OpenPrice'], mode='lines'))
    fig.update_layout(
        title='Cours du temps pour ' + paire,
        xaxis_title='Temps',
        yaxis_title='Prix d\'ouverture',
        yaxis_type='log'  # Change l'échelle de l'axe des y en une échelle logarithmique
    )

    return fig

# -->
def Alim_Dash_Data(paire, Periode_Debut, Periode_Fin, dureeJr_Entrainement = 90, ResetDB = False):
    
    try : 
        # --
        if ResetDB : 
            print('########Reinit Database#######')
            Bot.Reset_DB_All()

        # --
        StartTime = (datetime.fromisoformat(Periode_Debut) - timedelta(days = dureeJr_Entrainement)).strftime('%Y-%m-%d')
        EndTime = Periode_Debut
        
        
        print('#######Load DB Histo #######')
        Bot.Load_DB_Mongo([paire], StartTime, EndTime)
        Bot.Load_DB_SQL_Histo([paire], StartTime, EndTime)
        
        print('#######Load DB Live #######')
        Bot.Load_DB_SQL_Live([paire], EndTime, Periode_Fin)
        
        
        #--
        L = list()
        for Methode in ['M1', 'M2']:
            print('#######Etape Methode : {} #######'.format(Methode))
            
            #--
            print('#######Calcul Prediction Vente Achat #######')
            Bot.Load_DB_SQLPrediction(paire, Methode)
            X = Bot.Get_DataPaire([paire], Periode_Debut, Periode_Fin, Methode )           
            
            
            #--
            print('#######Generation Graphe Prediction Vente Achat #######')
            fig_DEC = Bot.Get_Graphe_Prediction_Achat_Vente(X)
            
            
            #--
            print('#######Generation Graphe Simulation #######')
            (rapport, Data) = Bot.Get_SimulationGain(X, paire)
            fig_GAIN = Bot.Get_Graphe_SimulationGain(Data)
            
            
            #--
            X = rapport.to_dict('records')
            
            X_temps = pd.DataFrame({ 'Information' : [i for i in X[0].keys()], 
                               'Value' : [X[0][i] for i in X[0].keys()] 
                             })
            L.append({'Methode' : Methode,
                      'Resultat' : {'rapport' : X_temps.to_dict('records'), 
                                    'fig_Graphe_Decision' : fig_DEC,
                                    'fig_Graphe_SimiGain' : fig_GAIN}
                     })

        
    except : 
        return [{}]
    
    return L


## Page 1 : Index

In [3]:
# Step 2 : Lancement du Dash Index appelant ces fonctions externes

# Build App
app = JupyterDash(external_stylesheets=[dbc.themes.CERULEAN])

app.layout = html.Div([
    html.H1("Test Graphique Acceuil"),
    html.Label([
        "Liste des Cryptos-Paire chez Binance",
        dcc.Dropdown(
            id='listePaire', clearable=False,
            value='ETHUSDT', options=Get_Dash_ListePaire())
    ]),
    dcc.Graph(id='graph'),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("listePaire", "value")]
)
def update_figure(paire):
    return Get_Dash_GraphAcceuil(paire, '2020-01-01', '2020-05-01')

# Run app and display result inline in the notebook
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



## Page 2 : Simulation Gain

In [4]:
#################################################
# 1ere Partie de la page #
#################################################

# --
container_Form = html.Div(
    [
        dbc.Row([
            
            
            dbc.Col(html.Label([
                        "Liste des Cryptos-Paire chez Binance",
                        dcc.Dropdown( id='paire-input', clearable=False, value='ETHUSDT', options=Get_Dash_ListePaire())      
                    ])),
        ]),
        
        html.Br(),
        dbc.Row(
            [
                dbc.Col( [ 
                            html.Label('Date de début (aaaa-mm-jj) :') , 
                            dcc.Input( id="date-debut-input", type="text", placeholder="2017-08-01" )
                         ] ),
                
                dbc.Col([
                            html.Label("Date de fin (aaaa-mm-jj) :"),
                            dcc.Input(id="date-fin-input", type="text", placeholder="2023-05-01")
                        ]),
                
                dbc.Col([
                            html.Label("Capital en dollar :"),
                            dcc.Input(id="capital-input", type="number", placeholder="10000"),
                        ]),
                
                dbc.Col( [ 
                            html.Br(),
                            dbc.Button("Lancement Simulation", id="Bouton_Backtest", n_clicks=0, outline=True, color="primary")
                         ])
                
            ]
        ),
        dbc.Alert(
            "Info : Fin Alimentation DataBase",
            id="alert-auto",
            is_open=False,
            dismissable=True,
        )
        
    ]
)
    

In [5]:
##### # 2e Partie de la page #
#################################################
container_Graph = html.Div(
    [
        dbc.Row(html.Div(
            [
                dbc.RadioItems(
                    id="radioItem_Methode",
                    options=[
                        {"label": "Méthode 1", "value": "M1"},
                        {"label": "Méthode 2", "value": "M2"},
                    ],
                    value="M1",
                    inline=True,
                ),
            ])),
        
        dbc.Row(
            [
                dbc.Col(dcc.Graph( id = 'grap_Decision' , )),
                dbc.Col(dcc.Graph( id = 'graph_Simulation',  ))
            ]
        ),
        dbc.Row(html.Div('Description Simulation Gain')),
        dbc.Row(html.Div(dash_table.DataTable(
            id ='Table_Rapport',
            style_cell={'textAlign': 'left'},
            columns=[{"name": i, "id": i} for i in ['Information', 'Value']]))
               ),
    ],
    id ="container_graph",
    hidden = False 
)

In [6]:

app = JupyterDash(external_stylesheets=[dbc.themes.CERULEAN])

# --
Data = Alim_Dash_Data("ETHUSDT", "2019-01-01", "2019-03-01", 90, True )



#################################################
# Main #
#################################################
app.layout = html.Div(
                [
                    html.H2("Analyse de performance du bot"),
                    container_Form, 
                    container_Graph,
                ]
            )



# --
@app.callback(
    Output('alert-auto', 'is_open'),
    
    Input("Bouton_Backtest", "n_clicks"),
    
    State("paire-input", "value"),
    State("date-debut-input", "value"),
    State("date-fin-input", "value"),
    State("capital-input", "value"),
         
    
)
def button_click (n_clicks, paire, DateDeb, DateFin, Capital):
    global Data
    
    if n_clicks != 0:
        Data = Alim_Dash_Data(paire, DateDeb, DateFin, 90, False )
        return True

# --
@app.callback(
    
    Output('grap_Decision', 'figure'),
    Output('graph_Simulation', 'figure'),
    Output('Table_Rapport', 'data'),
    Input("radioItem_Methode", "value"),
         
    
)
def Affiche_Graphe (Methode):
    global Data
    
    for resultat in Data:
        if resultat['Methode'] == Methode:
            rapport = resultat['Resultat']['rapport']
            fig_Dec = resultat['Resultat']['fig_Graphe_Decision']
            fig_Simu = resultat['Resultat']['fig_Graphe_SimiGain']
            return fig_Dec, fig_Simu, rapport
    

# Run app and display result inline in the notebook
app.run_server(mode='inline')

########Reinit Database#######
Step 1 : suppression de la base SQLlite
Step 2 : suppression des Collections  dans la Base Mongo
#######Load DB Histo #######
Récuparation de la Paire ETHUSDT à partir du 2020-01-01 jusquau 2020-05-01 
Téléchargement des Fichiers à charger
Chargement Base MongoDB
Connexion aux DataBases
Chargement Dimension Temps
Chargement Dimension Symbol
Chargement Fait Cours Historique
Récupération des Données des Paires ['ETHUSDT'] sur la période du 2018-10-03 au 2019-01-01


/home/arnold/Projet_OPA/Binance/Utils/Technical_Analyst.py:94: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/arnold/Projet_OPA/Binance/Utils/Technical_Analyst.py:95: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Lancement Fonction Boucle Trading M1
Chargement Desision Histo Achat Vente M1
Chargement Decision Histo Achat Vente M2
#######Load DB Live #######
Récuparation de la Paire ETHUSDT à partir du 2019-01-01 jusquau 2019-03-01 
Chargement Dimension Temps
Chargement Dimension Symbol
Chargement Fait Cours
#######Etape Methode : M1 #######
#######Calcul Prediction Vente Achat #######
Lancement Fonction Boucle Trading M1
Chargement Prediction Live Achat Vente M1
Récupération des Données des Paires ['ETHUSDT'] sur la période du 2019-01-01 au 2019-03-01
#######Generation Graphe Prediction Vente Achat #######
#######Generation Graphe Simulation #######
#######Etape Methode : M2 #######
#######Calcul Prediction Vente Achat #######


/home/arnold/Projet_OPA/Binance/Utils/Technical_Analyst.py:171: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Entrainement sur les 6 Derniers Mois
Prediction sur les données Lives
Optimisation Prediction Resultat
Chargement Prédiction Live Achat Vente M2
Récupération des Données des Paires ['ETHUSDT'] sur la période du 2019-01-01 au 2019-03-01
#######Generation Graphe Prediction Vente Achat #######
#######Generation Graphe Simulation #######
Dash is running on http://127.0.0.1:8050/



...

/home/arnold/Projet_OPA/Binance/Utils/Technical_Analyst.py:94: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/arnold/Projet_OPA/Binance/Utils/Technical_Analyst.py:95: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Chargement Decision Histo Achat Vente M2
#######Load DB Live #######
Récuparation de la Paire BTCUSDT à partir du 2020-01-01 jusquau 2022-01-01 
Chargement Dimension Temps
Chargement Dimension Symbol
Chargement Fait Cours
#######Etape Methode : M1 #######
#######Calcul Prediction Vente Achat #######
Lancement Fonction Boucle Trading M1
Chargement Prediction Live Achat Vente M1
Récupération des Données des Paires ['BTCUSDT'] sur la période du 2020-01-01 au 2022-01-01
#######Generation Graphe Prediction Vente Achat #######
#######Generation Graphe Simulation #######


/home/arnold/Projet_OPA/Binance/Utils/Technical_Analyst.py:171: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#######Etape Methode : M2 #######
#######Calcul Prediction Vente Achat #######
Entrainement sur les 6 Derniers Mois
Prediction sur les données Lives
Optimisation Prediction Resultat
Chargement Prédiction Live Achat Vente M2
Récupération des Données des Paires ['BTCUSDT'] sur la période du 2020-01-01 au 2022-01-01
#######Generation Graphe Prediction Vente Achat #######
#######Generation Graphe Simulation #######


/home/arnold/Projet_OPA/Binance/Utils/Technical_Analyst.py:171: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

